## Database fields comparison

In [1]:
import numpy as np
import datajoint as dj
import pandas as pd
import pygsheets
import json
import difflib
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
gc = pygsheets.authorize(service_file='u19datajoint-b121ee91f642.json')

### Connect to database

In [2]:
dj.config['database.host'] = 'datajoint01.pni.princeton.edu'
#dj.config['database.user'] = 'alvaros'

conn = dj.conn(host=dj.config['database.host'], user=dj.config['database.user'])


Please enter DataJoint username: alvaros
Please enter DataJoint password: ········
Connecting alvaros@datajoint01.pni.princeton.edu:3306


### Initial parameters

In [3]:
#Which tables are supposed to be copy
tables_copy = [
    'schedule',
'water',
'mass',
'rigwater',
'technotes',
'rat_history',
'rigflush',
'phys',
'rig_maintenance',
'rats',
'tech_schedule',
'turn_down_log',
'rigfood',
'surgery',
'rigvideo',
'cerebro_sessions',
'contacts',
'riginfo',
'eibs',
'training_room',
'rigtrials',
'infusions',
'parsed_events',
'sessions',
'spktimes',
'channels',
'sess_started',
'sess_list',
'calibration_info_tbl',
'cells',
'phys_sess',
'pbups']


#Custom order for tables in dataframe
table_order = [
'contacts',
'riginfo',
'rig_maintenance',
'rigflush',
'rigfood',
'training_room',
'rats',
'rat_history',
'schedule',
'tech_schedule',
'mass',
'water',
'calibration_info_tbl',
'rigvideo',
'rigwater',
'technotes',
'surgery',
'sess_started',
'sessions',
'infusions',
'cerebro_sessions',
'phys_sess',
'channels',
'cells',
'spktimes',
'eibs',
'turn_down_log',
'pbups']

fields_order = ['NaN']

#Custom order of columns in dataframe
column_order = ["TABLE_SCHEMA", "TABLE_NAME", "COLUMN_NAME", "COLUMN_TYPE",
                "NEW_TABLE_SCHEMA", "NEW_TABLE_NAME", "NEW_COLUMN_NAME", "NEW_COLUMN_TYPE"]


schemas_compare = [
    'bl_ratinfo',
    'bl_bdata',
    'bl_protocol',
    'bl_new_lab',
    'bl_new_subject',
    'bl_action',
    'bl_acquisition',
    'bl_ephys',
    'bl_eibs',
    'bl_pbups_behavior',
]

#Start row for borders (e.g A2)
start_row_index = 2

### All table dataframe

In [4]:
#Overview across database schemas
query_tables = conn.query('select TABLE_SCHEMA, TABLE_NAME from information_schema.tables where TABLE_SCHEMA LIKE %s', 
                          ("%" + "bl_" + "%",))
dftables = pd.DataFrame(query_tables.fetchall())
dftables.columns = [x[0] for x in query_tables.description ]

dftables = dftables.loc[dftables['TABLE_SCHEMA'].isin(schemas_compare), :]
dftables = dftables.reset_index(drop=True)
dftables


,TABLE_SCHEMA,TABLE_NAME
0,bl_acquisition,cerebro_sessions
1,bl_acquisition,infusions
2,bl_acquisition,sess_started
3,bl_acquisition,sessions
4,bl_action,calibration_info_tbl
...,...,...
97,bl_ratinfo,turn_down_log
98,bl_ratinfo,video_log
99,bl_ratinfo,videoinfo
100,bl_ratinfo,water


### All column dataframe

In [5]:
query_all_column = conn.query('select TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, COLUMN_TYPE from information_schema.columns where TABLE_SCHEMA LIKE %s', 
                          ("%" + "bl_" + "%",))

df_columns = pd.DataFrame(query_all_column.fetchall())
df_columns.columns = [x[0] for x in query_all_column.description ]

df_columns = df_columns.loc[df_columns['TABLE_SCHEMA'].isin(schemas_compare), :]
df_columns = df_columns.reset_index(drop=True)

df_columns


,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,COLUMN_TYPE
0,bl_acquisition,cerebro_sessions,sessid,int(11)
1,bl_acquisition,cerebro_sessions,cerebro_session_date,date
2,bl_acquisition,cerebro_sessions,exclude,tinyint(1)
3,bl_acquisition,cerebro_sessions,exclude_reason,varchar(1000)
4,bl_acquisition,cerebro_sessions,comments,varchar(1000)
...,...,...,...,...
2039,bl_ratinfo,~log,timestamp,timestamp
2040,bl_ratinfo,~log,version,varchar(12)
2041,bl_ratinfo,~log,user,varchar(255)
2042,bl_ratinfo,~log,host,varchar(255)


### Merge old and new schemas table dataframe

In [6]:
#Select old tables schemas
old_schemas = ['bl_ratinfo', 'bl_bdata', 'bl_protocol'];
df_oldtables =  dftables[dftables['TABLE_SCHEMA'].isin(old_schemas)]
df_newtables =  dftables[~dftables['TABLE_SCHEMA'].isin(old_schemas)]

#Rewname columns for new tables dataframe
df_newtables = df_newtables.rename(columns={"TABLE_SCHEMA": "NEW_TABLE_SCHEMA", "TABLE_NAME": "NEW_TABLE_NAME"})

#Merge dataframes
df_tables_merged = df_oldtables.merge(df_newtables, left_on='TABLE_NAME', right_on='NEW_TABLE_NAME')
df_tables_merged

#Sort dataframe by custom table order defined at the beginning
df_tables_merged["TABLE_NAME"] = df_tables_merged["TABLE_NAME"].astype("category")
df_tables_merged["TABLE_NAME"] = df_tables_merged["TABLE_NAME"].cat.set_categories(table_order)

df_tables_merged = df_tables_merged.sort_values(["TABLE_NAME"])
df_tables_merged = df_tables_merged.reset_index(drop =True)

### Create sheet for column comparison

In [7]:
#Open SpreadSheet for column comparison
sh = gc.open_by_key('1rbQ3hSzW3Qc_1Qkx8XyVpVoMasSveooDqhyOT3AheQI')
wks = sh.sheet1


#Clean comparison sheet and create new one
try:
    wks_comparison = sh.worksheet_by_title('ColumnComparison')
    sh.del_worksheet(wks_comparison)
    print('Deleting previous ColumnComparison')
except:
    print('No previous ColumnComparison')
    
sh.add_worksheet('ColumnComparison', 
                 src_tuple=('1AmACeNqIyUWrSsYChRN2_jS2k1zzo2_m_PbGtcPMX44', 1022926331), index=0)

wks_comparison = sh.worksheet('title', 'ColumnComparison')



Deleting previous ColumnComparison


### Generate dataframe to compare columns name for each table

In [8]:
#Create dataframe for all columns and auxiliar dropdown dataframe
df_all_columns_merged = pd.DataFrame([])
df_dropdown_guide = pd.DataFrame(columns=['Start', 'End', 'ListValues', 
                                          'StartRange', 'EndRange'])


index_row = start_row_index
# For all tables to compare
for i in range(df_tables_merged.shape[0]):
    
    # Get old and new tables with column (field) names
    
    df_old_table = df_columns[(df_columns['TABLE_SCHEMA'] ==  df_tables_merged.loc[i,"TABLE_SCHEMA"]) &
                              (df_columns['TABLE_NAME'] ==    df_tables_merged.loc[i,"TABLE_NAME"])]
    
    df_new_table = df_columns[(df_columns['TABLE_SCHEMA'] ==  df_tables_merged.loc[i,"NEW_TABLE_SCHEMA"]) &
                              (df_columns['TABLE_NAME'] ==    df_tables_merged.loc[i,"NEW_TABLE_NAME"])]
    
    # Rename the columns in the new table dataframe
    
    df_new_table = df_new_table.reset_index(drop = True)
    
    df_new_table = df_new_table.rename(columns={"TABLE_SCHEMA": "NEW_TABLE_SCHEMA", "TABLE_NAME": "NEW_TABLE_NAME",
                                                "COLUMN_NAME": "NEW_COLUMN_NAME"})
    
    # Drop column_type and reorder to easy manual comparison 
    df_old_table = df_old_table.drop(columns=['COLUMN_TYPE'])
    df_new_table = df_new_table.drop(columns=['COLUMN_TYPE'])
    df_new_table = df_new_table[["NEW_COLUMN_NAME", "NEW_TABLE_SCHEMA", "NEW_TABLE_NAME"]]
    
    # Merge tables to find matching fields between old and new tables
    df_columns_merged = df_old_table.merge(df_new_table, left_on=['TABLE_NAME', 'COLUMN_NAME'],
                                           right_on=['NEW_TABLE_NAME', 'NEW_COLUMN_NAME'],
                                           how='left', indicator=True)
    
    # Dismiss fields with same field names (no change)
    df_columns_merged = df_columns_merged.loc[df_columns_merged['_merge'] != 'both', :]
    df_columns_merged = df_columns_merged.reset_index(drop = True)
    df_columns_merged = df_columns_merged.drop(columns=['_merge'])

    
    # Get field names of non matching fields of new tables and save them to a list
    df_new_columns_no_match = df_old_table.merge(df_new_table, left_on=['TABLE_NAME', 'COLUMN_NAME'],
                                                 right_on=['NEW_TABLE_NAME', 'NEW_COLUMN_NAME'], 
                                                 how='right', indicator=True)
    
    df_new_columns_no_match = df_new_columns_no_match[df_new_columns_no_match['_merge'] == 'right_only']
    list_non_match = df_new_columns_no_match['NEW_COLUMN_NAME'].values.tolist();
    

    # Remove NaN from NEW_TABLE_SCHEMA and NEW_TABLE_NAME columns
    df_columns_merged.loc[:,'NEW_TABLE_SCHEMA'] = df_new_table.loc[0, 'NEW_TABLE_SCHEMA']
    df_columns_merged.loc[:,'NEW_TABLE_NAME'] = df_new_table.loc[0, 'NEW_TABLE_NAME']
        
    # Fill df_dropdown_guide dataframe (dropwdown list with possible options)
    idx_null = np.flatnonzero(df_columns_merged['NEW_COLUMN_NAME'].isnull())
    idx_col_new_column_name = df_columns_merged.columns.get_loc('NEW_COLUMN_NAME')
    if len(idx_null) > 0:
        #Generate string that defines cellrange (for dropdown list)
        start_string = chr(65+idx_col_new_column_name)+str(idx_null[0]+index_row)
        end_string = chr(65+idx_col_new_column_name)+str(idx_null[-1]+index_row)
        #Generate cellrange for border drawing
        start_range_string = 'A'+str(idx_null[0]+index_row)
        end_range_string = chr(65+df_columns_merged.shape[1]-1)+str(idx_null[-1]+index_row)

        new_row = {'Start':start_string, 'End':end_string, 'ListValues':['NaN'] + list_non_match,
                  'StartRange':start_range_string, 'EndRange': end_range_string}
        df_dropdown_guide = df_dropdown_guide.append(new_row, ignore_index=True)
        index_row = index_row + idx_null[-1] + 1
        
    # Fill closest match on non matching columns
    for j in range(df_columns_merged.shape[0]):
        value_match = df_columns_merged.loc[j, "COLUMN_NAME"].lower()
        # Close match
        close_match = difflib.get_close_matches(value_match, list_non_match, n=1, cutoff=0.6)
        
        # Match inside new name (e.g. id. -> old_id  is a match inside)
        inside_match = [s for s in list_non_match if value_match in s]
        
        # Exact match
        if value_match in list_non_match:
            df_columns_merged.loc[j, "NEW_COLUMN_NAME"] = value_match
        elif len(inside_match) > 0:
            df_columns_merged.loc[j, "NEW_COLUMN_NAME"] = inside_match[0]
        elif len(close_match) > 0:
            df_columns_merged.loc[j, "NEW_COLUMN_NAME"] = close_match[0]

            
    #Concatenate in main dataframe
    df_all_columns_merged = pd.concat([df_all_columns_merged, df_columns_merged])

    

df_all_columns_merged = df_all_columns_merged.reset_index(drop = True)
df_dropdown_guide


,Start,End,ListValues,StartRange,EndRange
0,D2,D7,"[NaN, user_id, contacts_old_id, lab_manage, fu...",A2,F7
1,D8,D9,"[NaN, rigname]",A8,F9
2,D10,D13,"[NaN, rig_maintenance_id, rig_fix_date, rig_ma...",A10,F13
3,D14,D16,"[NaN, rigid, rigflush_date, rigflush_old_id]",A14,F16
4,D17,D18,"[NaN, rigfood_id, rigfood_datetime]",A17,F18
5,D19,D30,"[NaN, user_id, rats_old_id, force_free_water, ...",A19,F30
6,D31,D37,"[NaN, user_id, rathistory_old_id, force_free_w...",A31,F37
7,D38,D45,"[NaN, schedule_date, schedule_timeslot, user_i...",A38,F45
8,D46,D46,"[NaN, tech_schedule_date]",A46,F46
9,D47,D50,"[NaN, weighing_date, weighing_time, weigh_pers...",A47,F50


### Write dataframe in Spreadsheet for manual column comparison (pygsheets set_data_validation needed !!)

## Insert this code in your virtual environment site-packages/pygsheets/worksheet.py  (line 1278)

    @batchable
    def set_data_validation(self, start=None, end=None, condition_type=None, condition_values=None,
                            grange=None, **kwargs):
        """
        Sets a data validation rule to every cell in the range. To clear validation in a range,
        call this with no condition_type specified.

        refer to `api docs <https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets/other#conditiontype>`__ for possible inputs.

        :param start: start address
        :param end: end address
        :param grange: address as grid range
        :param condition_type: validation condition type: `possible values <https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets/other#conditiontype>`__
        :param condition_values: list of values for supporting condition type. For example ,
                when condition_type is NUMBER_BETWEEN, value should be two numbers indicationg lower
                and upper bound. See api docs for more info.
        :param kwargs: other options of rule.
                possible values: inputMessage, strict, showCustomUi
                `ref <https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets/cells#datavalidationrule>`__
        """
        if not grange:
            grange = GridRange(worksheet=self, start=start, end=end)
        grange.set_worksheet(self)

        condition_values = list() if not condition_values else condition_values
        json_values = []
        for value in condition_values:
            if condition_type in \
                    ['DATE_BEFORE', 'DATE_AFTER', 'DATE_ON_OR_BEFORE', 'DATE_ON_OR_AFTER']:
                json_values.append({'relativeDate': str(value)})
            else:
                json_values.append({'userEnteredValue': str(value)})

        request = {"setDataValidation": {
            "range": grange.to_json()
        }
        }
        if condition_type:
            rule = {'condition': {
                'type': condition_type,
                'values': json_values
            }
            }
            for kwarg in kwargs:
                rule[kwarg] = kwargs[kwarg]
            request['setDataValidation']['rule'] = rule
        self.client.sheet.batch_update(self.spreadsheet.id, request)


In [9]:
# Write main datafrmae
wks_comparison.set_dataframe(df_all_columns_merged,'A1')

# Write dropdown options
for i in range(df_dropdown_guide.shape[0]):
    wks_comparison.set_data_validation(start=df_dropdown_guide.loc[i, 'Start'], end=df_dropdown_guide.loc[i, 'End'], 
                            condition_type='ONE_OF_LIST', condition_values=df_dropdown_guide.loc[i, 'ListValues'],
                            strict=True, showCustomUi=True)
    
    drange = pygsheets.datarange.DataRange(start=df_dropdown_guide.loc[i, 'StartRange'], 
                                           end=df_dropdown_guide.loc[i, 'EndRange'], worksheet=wks_comparison)
    drange.update_borders(top=True, right=True, bottom=True, left=True, style='SOLID', width=2)

wks_comparison.index = 0

## Get sheet with corrected columns (after manual comparison)

In [13]:
# Get corrected columns sheet
wks_corrected_columns = sh.worksheet_by_title('CorrectedColumns')
df_corrected_columns = wks_corrected_columns.get_as_df()
df_corrected_columns

,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,NEW_COLUMN_NAME,NEW_TABLE_SCHEMA,NEW_TABLE_NAME
0,bl_ratinfo,contacts,tag_RGB_old,NaN,bl_new_lab,contacts
1,bl_ratinfo,contacts,lab_manager,lab_manage,bl_new_lab,contacts
2,bl_ratinfo,contacts,custom_rig_order,NaN,bl_new_lab,contacts
3,bl_ratinfo,contacts,FullName,full_name,bl_new_lab,contacts
4,bl_ratinfo,contacts,tag_RGB,tag_rgb,bl_new_lab,contacts
...,...,...,...,...,...,...
198,bl_protocol,pbups,PenaltySection_LEDtemp_pun,penaltysection_ledtemp_pun,bl_pbups_behavior,pbups
199,bl_protocol,pbups,PenaltySection_SideChoicePunishmentType,penaltysection_sidechoicepunishmenttype,bl_pbups_behavior,pbups
200,bl_protocol,pbups,PenaltySection_PunishSideChoice,penaltysection_punishsidechoice,bl_pbups_behavior,pbups
201,bl_protocol,pbups,PenaltySection_PunishSidePokesinWFCO,penaltysection_punishsidepokesinwfco,bl_pbups_behavior,pbups


### Append missing new columns (extra fields on new tables)

In [14]:

#Copy dataframe to correct final stuff
df_corrected_columns_final = df_corrected_columns.copy()
df_border_guide = df_dropdown_guide.copy()
index_row = start_row_index

# For each table on the databases
for i in range(df_dropdown_guide.shape[0]):
    
    #Get start and end row on the corrected columns dataframe
    start_row = int(df_dropdown_guide.loc[i, 'StartRange'][1:])-start_row_index
    end_row = int(df_dropdown_guide.loc[i, 'EndRange'][1:])-start_row_index
    
    #New columns (non match) on that table
    all_new_columns = pd.Series(df_dropdown_guide.loc[i, 'ListValues'])
    
    #Get all fields that weren't matched to pairs on the old tables
    non_matched_fields = all_new_columns[~all_new_columns.isin(df_corrected_columns.loc[start_row:end_row, 'NEW_COLUMN_NAME'])]

    #Remove NaN
    non_matched_fields = non_matched_fields[~non_matched_fields.isin(['NaN'])]

    #Create a dataframe to append non matched fields
    if non_matched_fields.shape[0] > 0:
    
        #Auxiliar dataframe with same columns and repeated N times (n non matching fields)
        dfaux = df_corrected_columns.loc[end_row, :].to_frame().transpose()
        df_non_matched_fields = dfaux.append([dfaux]*(non_matched_fields.shape[0]-1),ignore_index=True)

        #Fill values for column name and new_column_name
        df_non_matched_fields['COLUMN_NAME'] = 'NaN'
        df_non_matched_fields['NEW_COLUMN_NAME'] = non_matched_fields.tolist()
        
        #Append it to the main dataframe
        df_corrected_columns_final = df_corrected_columns_final.append(df_non_matched_fields,ignore_index=True)
        extra_rows_ = df_non_matched_fields.shape[0]
    else:
        extra_rows_ = 0 
    
    #Recalculate Start and EndRange (for border drawing)
    size_fields_table = end_row-start_row
    cols_ = df_corrected_columns_final.shape[1]
    df_border_guide.loc[i, 'StartRange'] = 'A'+str(index_row)
    df_border_guide.loc[i, 'EndRange'] = 'K' + str(index_row+size_fields_table+extra_rows_)

    index_row = index_row + (size_fields_table) + extra_rows_ + 1

    




### Add datatypes to our main dataframe

In [15]:
# Get old and new tables with column (field) names
df_final_order = df_corrected_columns_final.copy()

df_datatypes = df_columns.loc[:,['TABLE_SCHEMA','TABLE_NAME', 'COLUMN_NAME', 'COLUMN_TYPE']]

#Get column_type of old tables
df_final_order = df_final_order.merge(df_datatypes, left_on=['TABLE_SCHEMA', 'TABLE_NAME', 'COLUMN_NAME'],
                                                   right_on=['TABLE_SCHEMA', 'TABLE_NAME', 'COLUMN_NAME'], 
                                                   how='left')

#Rename datatype_df to merge easily 
df_datatypes = df_datatypes.rename(columns={"TABLE_SCHEMA":"NEW_TABLE_SCHEMA", "TABLE_NAME":"NEW_TABLE_NAME",
                                            "COLUMN_NAME": "NEW_COLUMN_NAME", "COLUMN_TYPE": "NEW_COLUMN_TYPE"})

#Get column_type of new tables
df_final_order = df_final_order.merge(df_datatypes, left_on=['NEW_TABLE_SCHEMA', 'NEW_TABLE_NAME', 'NEW_COLUMN_NAME'],
                                                    right_on=['NEW_TABLE_SCHEMA', 'NEW_TABLE_NAME', 'NEW_COLUMN_NAME'], 
                                                    how='left')

#Reorder columns for spreadsheet
df_final_order = df_final_order[column_order]


### Sort DataFrame by Table and NaN fields first

In [16]:
#Table name as categpry
df_final_order["TABLE_NAME"] = df_final_order["TABLE_NAME"].astype("category")
df_final_order["TABLE_NAME"] = df_final_order["TABLE_NAME"].cat.set_categories(table_order)

# Column name as category (Nan goes first)
df_final_order["COLUMN_NAME"] = df_final_order["COLUMN_NAME"].astype("category")
all_column_names = df_corrected_columns_final["COLUMN_NAME"].copy()
all_column_names = all_column_names[~all_column_names.isin(['NaN'])]
column_name_order = fields_order + pd.unique(all_column_names).tolist()
df_final_order["COLUMN_NAME"] = df_final_order["COLUMN_NAME"].cat.set_categories(column_name_order)


df_final_order.loc[df_final_order["NEW_COLUMN_NAME"].isnull(),"NEW_COLUMN_NAME"] = "NaN" 

#Sort all 
df_final_order = df_final_order.sort_values(["TABLE_NAME", "COLUMN_NAME", "NEW_COLUMN_NAME"])
df_final_order["TABLE_NAME"] = df_final_order["TABLE_NAME"].astype("object")
df_final_order["COLUMN_NAME"] = df_final_order["COLUMN_NAME"].astype("object")
df_final_order = df_final_order.reset_index(drop =True)
df_final_order

,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,COLUMN_TYPE,NEW_TABLE_SCHEMA,NEW_TABLE_NAME,NEW_COLUMN_NAME,NEW_COLUMN_TYPE
0,bl_ratinfo,contacts,NaN,NaN,bl_new_lab,contacts,user_id,varchar(32)
1,bl_ratinfo,contacts,tag_RGB_old,tinyblob,bl_new_lab,contacts,NaN,NaN
2,bl_ratinfo,contacts,lab_manager,tinyint(1),bl_new_lab,contacts,lab_manage,tinyint(1)
3,bl_ratinfo,contacts,custom_rig_order,varchar(128),bl_new_lab,contacts,NaN,NaN
4,bl_ratinfo,contacts,FullName,varchar(60),bl_new_lab,contacts,full_name,varchar(60)
...,...,...,...,...,...,...,...,...
213,bl_protocol,pbups,PenaltySection_LEDtemp_pun,float,bl_pbups_behavior,pbups,penaltysection_ledtemp_pun,float
214,bl_protocol,pbups,PenaltySection_SideChoicePunishmentType,float,bl_pbups_behavior,pbups,penaltysection_sidechoicepunishmenttype,float
215,bl_protocol,pbups,PenaltySection_PunishSideChoice,tinyint(1),bl_pbups_behavior,pbups,penaltysection_punishsidechoice,tinyint(1)
216,bl_protocol,pbups,PenaltySection_PunishSidePokesinWFCO,float,bl_pbups_behavior,pbups,penaltysection_punishsidepokesinwfco,float


### Write final dataframe in CorrectedColumnsFinal

In [17]:

#Clean comparison sheet and create new one
try:
    wks_comparison = sh.worksheet_by_title('CorrectedColumnsFinal')
    sh.del_worksheet(wks_comparison)
    print('Deleting previous CorrectedColumnsFinal')
except:
    print('No previous CorrectedColumnsFinal')
    
sh.add_worksheet('CorrectedColumnsFinal', src_tuple=('1AmACeNqIyUWrSsYChRN2_jS2k1zzo2_m_PbGtcPMX44', 177458389))
wks_corrected_final = sh.worksheet('title', 'CorrectedColumnsFinal')

# Write main datafrmae
wks_corrected_final.set_dataframe(df_final_order,'A1')

# Write border options
for i in range(df_border_guide.shape[0]):
    
    drange = pygsheets.datarange.DataRange(start=df_border_guide.loc[i, 'StartRange'], 
                                           end=df_border_guide.loc[i, 'EndRange'], worksheet=wks_corrected_final)
    drange.update_borders(top=True, right=True, bottom=True, left=True, style='SOLID', width=2)



Deleting previous CorrectedColumnsFinal


### Dataframe for same field names, different tpyes

In [18]:
#Create dataframe for all columns and auxiliar dropdown dataframe
df_same_field_different_type = pd.DataFrame([])
df_border_guide_diff = pd.DataFrame(columns=['StartRange', 'EndRange'])

index_row = start_row_index

# For all tables to compare
for i in range(df_tables_merged.shape[0]):
    
    # Get old and new tables with column (field) names
    
    df_old_table = df_columns[(df_columns['TABLE_SCHEMA'] ==  df_tables_merged.loc[i,"TABLE_SCHEMA"]) &
                              (df_columns['TABLE_NAME'] ==    df_tables_merged.loc[i,"TABLE_NAME"])]
    
    df_new_table = df_columns[(df_columns['TABLE_SCHEMA'] ==  df_tables_merged.loc[i,"NEW_TABLE_SCHEMA"]) &
                              (df_columns['TABLE_NAME'] ==    df_tables_merged.loc[i,"NEW_TABLE_NAME"])]
    
    # Rename the columns in the new table dataframe
    
    df_new_table = df_new_table.reset_index(drop = True)
    
    df_new_table = df_new_table.rename(columns={"TABLE_SCHEMA": "NEW_TABLE_SCHEMA", "TABLE_NAME": "NEW_TABLE_NAME",
                                                "COLUMN_NAME": "NEW_COLUMN_NAME", "COLUMN_TYPE": "NEW_COLUMN_TYPE"})
    
    
    # Merge tables to find matching fields between old and new tables
    df_columns_merged = df_old_table.merge(df_new_table, left_on=['TABLE_NAME', 'COLUMN_NAME'],
                                           right_on=['NEW_TABLE_NAME', 'NEW_COLUMN_NAME'],
                                           how='left', indicator=True)
    
    # Get only fields with same field names
    df_columns_merged = df_columns_merged.loc[df_columns_merged['_merge'] == 'both', :]
    df_columns_merged = df_columns_merged.reset_index(drop = True)
    df_columns_merged = df_columns_merged.drop(columns=['_merge'])
    
    # Dismiss fields with same data type
    df_columns_merged = df_columns_merged.loc[(df_columns_merged['COLUMN_TYPE'] !=  df_columns_merged['NEW_COLUMN_TYPE']),:]

    
    #Concatenate in main dataframe
    if df_columns_merged.shape[0] > 0:
        df_same_field_different_type = pd.concat([df_same_field_different_type, df_columns_merged])

        #Border drawing dataframe
        start_range_string = 'A'+str(index_row)
        end_range_string = 'D' + str(df_columns_merged.shape[0]+index_row-1)

        new_row = {'StartRange':start_range_string, 'EndRange': end_range_string}
        df_border_guide_diff = df_border_guide_diff.append(new_row, ignore_index=True)
        index_row = index_row + df_columns_merged.shape[0]
    
    
#Rest index and remove not necessary columns    
df_same_field_different_type = df_same_field_different_type.reset_index(drop=True)    
    
df_same_field_different_type = df_same_field_different_type[['TABLE_NAME', 'COLUMN_NAME', 'COLUMN_TYPE', 
                                                                                     'NEW_COLUMN_TYPE']]

    

### Write dataframe for different types

In [19]:

#Clean comparison sheet and create new one
try:
    wks_comparison = sh.worksheet_by_title('SameField_DifType')
    sh.del_worksheet(wks_comparison)
    print('Deleting previous SameField_DifType')
except:
    print('No previous SameField_DifType')
    
sh.add_worksheet('SameField_DifType', src_tuple=('1AmACeNqIyUWrSsYChRN2_jS2k1zzo2_m_PbGtcPMX44', 858161751))
wks_diff_types = sh.worksheet('title', 'SameField_DifType')

# Write main datafrmae
wks_diff_types.set_dataframe(df_same_field_different_type,'A1')

# Write border options
for i in range(df_border_guide_diff.shape[0]):
    
    drange = pygsheets.datarange.DataRange(start=df_border_guide_diff.loc[i, 'StartRange'], 
                                           end=df_border_guide_diff.loc[i, 'EndRange'], worksheet=wks_diff_types)
    drange.update_borders(top=True, right=True, bottom=True, left=True, style='SOLID', width=2)

Deleting previous SameField_DifType


### Check deleted tables from original ones

In [20]:

df_tables_2copy = df_oldtables.loc[df_oldtables['TABLE_NAME'].isin(tables_copy),:]

#Merge dataframe from old and new table
df_tables_not_copied = df_tables_2copy.merge(df_newtables, left_on='TABLE_NAME', right_on='NEW_TABLE_NAME',
                                          how='left', indicator=True)

#df_tables_not_copied = df_tables_not_copied[df_tables_not_copied['_merge'] == 'left_only']
df_tables_not_copied = df_tables_not_copied.drop(columns=['_merge'])


df_tables_not_copied.loc[df_tables_not_copied["NEW_TABLE_SCHEMA"].isnull(),"NEW_TABLE_SCHEMA"] = "NaN" 
# NEW_TABLE_SCHEMAas category (Nan goes first)
df_tables_not_copied["NEW_TABLE_SCHEMA"] = df_tables_not_copied["NEW_TABLE_SCHEMA"].astype("category")
all_column_names = df_corrected_columns_final["NEW_TABLE_SCHEMA"].copy()
all_column_names = all_column_names[~all_column_names.isin(['NaN'])]
column_name_order = fields_order + pd.unique(all_column_names).tolist()
df_tables_not_copied["NEW_TABLE_SCHEMA"] = df_tables_not_copied["NEW_TABLE_SCHEMA"].cat.set_categories(column_name_order)


df_tables_not_copied
df_tables_not_copied = df_tables_not_copied.sort_values(["NEW_TABLE_SCHEMA"])
df_tables_not_copied["NEW_TABLE_SCHEMA"] = df_tables_not_copied["NEW_TABLE_SCHEMA"].astype("object")
df_tables_not_copied = df_tables_not_copied.reset_index(drop =True)
df_tables_not_copied


,TABLE_SCHEMA,TABLE_NAME,NEW_TABLE_SCHEMA,NEW_TABLE_NAME
0,bl_ratinfo,phys,NaN,NaN
1,bl_bdata,parsed_events,NaN,NaN
2,bl_bdata,sess_list,NaN,NaN
3,bl_ratinfo,rigtrials,NaN,NaN
4,bl_ratinfo,training_room,bl_new_lab,training_room
5,bl_ratinfo,rig_maintenance,bl_new_lab,rig_maintenance
6,bl_ratinfo,rigflush,bl_new_lab,rigflush
7,bl_ratinfo,rigfood,bl_new_lab,rigfood
8,bl_ratinfo,riginfo,bl_new_lab,riginfo
9,bl_ratinfo,contacts,bl_new_lab,contacts


### Write dataframe of tables not copied

In [21]:

#Clean comparison sheet and create new one
try:
    wks_comparison = sh.worksheet_by_title('TablesNotCopied')
    sh.del_worksheet(wks_comparison)
    print('Deleting previous TablesNotCopied')
except:
    print('No previous TablesNotCopied')
    
sh.add_worksheet('TablesNotCopied', src_tuple=('1AmACeNqIyUWrSsYChRN2_jS2k1zzo2_m_PbGtcPMX44', 165158746))
wks_diff_types = sh.worksheet('title', 'TablesNotCopied')

# Write main datafrmae
wks_diff_types.set_dataframe(df_tables_not_copied,'A1')


Deleting previous TablesNotCopied
